<a href="https://colab.research.google.com/github/Billy-Drunkenstein/MAFN/blob/main/Spring%202025/Machine%20Learning%20for%20Finance/Homework%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MATHGR5430 MACHINE LEARNING FOR FINANCE, SPRING 2025
<center>
<h1>Homework 3</h1>
</center>


## Background
You are a data scientist at a credit card company. A new dataset just landed in front of you. Your manager asked you to predict whether a client will make a large amount of payment(large 'payment' column). Before diving into modeling, you need to make the data usable. (Check datacard.txt) All the questions are based on this data set.

In [17]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity
import random

## Question 1

1.1 Data Cleaning

Import the dataset as a pandas dataframe. (Google is your best friend here, mount your google drive to colab and read your csv there)



In [2]:
data = pd.read_csv('HW3.csv')
data.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


Drop a irrelevant features (Hint: there is a column should never be use for any prediction and should not affect the prediction model, drop that one).


In [3]:
data = data.drop(columns = ['CUST_ID'])
data.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12



Identify and replace missing values (with any justifable approach to maintain datasize).



In [4]:
data = data.fillna(data.median(numeric_only=True))
data.isna().sum()

,0
BALANCE,0
BALANCE_FREQUENCY,0
PURCHASES,0
ONEOFF_PURCHASES,0
INSTALLMENTS_PURCHASES,0
CASH_ADVANCE,0
PURCHASES_FREQUENCY,0
ONEOFF_PURCHASES_FREQUENCY,0
PURCHASES_INSTALLMENTS_FREQUENCY,0
CASH_ADVANCE_FREQUENCY,0


Identify and replace outliers. Justify your approach.



In [5]:
# Capping at 1st and 99th percentile
for col in data.select_dtypes(include='number').columns:
    lower = data[col].quantile(0.01)
    upper = data[col].quantile(0.99)
    data[col] = data[col].clip(lower, upper)

1.2 Exploratory Data Analysis

Plot the correlation matrix. What relationships or multicollinearity do you observe?



In [6]:
corr_matrix = data.select_dtypes(include='number').corr()

fig = px.imshow(
    corr_matrix,
    color_continuous_scale='RdBu_r',
    title='Correlation Matrix',
    aspect='auto'
)

fig.update_traces(texttemplate=None)

fig.show()

## There are multiple high correlation pairs: (PURCHASES, ONEOFF_PURCHASES), (PURCHASES_TRX, PURCHASES), (PURCHASES_INSTALLMENTS_FREQUENCY, PURCHASES_FREQUENCY), (CASH_ADVANCE_TRX, CASH_ADVANCE_FREQUENCY)
## Pairs with high correlation appear to share merits within each individual. For example,  more transactions often mean higher cash advance amounts, leading to (CASH_ADVANCE_TRX, CASH_ADVANCE_FREQUENCY) pair having high correlation

Provide summary statistics and visualizations (e.g., histograms, boxplots) for key variables like PURCHASES, CREDIT_LIMIT, PAYMENTS, MINIMUM_PAYMENTS, etc.



In [7]:
# I would consider the following variables as important
key_variables = [
    "PURCHASES",
    "PURCHASES_FREQUENCY",
    "ONEOFF_PURCHASES_FREQUENCY",
    "CASH_ADVANCE",
    "CASH_ADVANCE_FREQUENCY",
    "CREDIT_LIMIT",
    "PRC_FULL_PAYMENT",
    "PAYMENTS",
    "MINIMUM_PAYMENTS",
    "BALANCE"
]

In [8]:
summary = data[key_variables].describe().T
summary

,count,mean,std,min,25%,50%,75%,max
PURCHASES,8950.0,936.229237,1525.185179,0.000000,39.635000,361.280000,1110.130000,8977.290000
PURCHASES_FREQUENCY,8950.0,0.490351,0.401371,0.000000,0.083333,0.500000,0.916667,1.000000
ONEOFF_PURCHASES_FREQUENCY,8950.0,0.202458,0.298336,0.000000,0.000000,0.083333,0.300000,1.000000
CASH_ADVANCE,8950.0,935.920972,1775.774147,0.000000,0.000000,0.000000,1113.821139,9588.163357
CASH_ADVANCE_FREQUENCY,8950.0,0.134017,0.195550,0.000000,0.000000,0.000000,0.222222,0.833333
CREDIT_LIMIT,8950.0,4474.254540,3551.099746,500.000000,1600.000000,3000.000000,6500.000000,17000.000000
PRC_FULL_PAYMENT,8950.0,0.153715,0.292499,0.000000,0.000000,0.000000,0.142857,1.000000
PAYMENTS,8950.0,1649.420364,2251.668769,0.000000,383.276166,856.901546,1901.134317,13608.715541
MINIMUM_PAYMENTS,8950.0,747.406471,1232.748218,20.040613,170.857654,312.343947,788.713501,8626.691541
BALANCE,8950.0,1541.046707,1966.597484,0.065101,128.281915,873.385231,2054.140036,9338.804814


In [9]:
for col in key_variables:
    fig = px.histogram(data, x=col, nbins=100, title=f'Histogram of {col}')
    fig.show()

Create a binary label HIGH_PAYMENT where:
HIGH_PAYMENT = 1 if PAYMENTS > 4000 else 0

In [10]:
data["HIGH_PAYMENT"] = (data["PAYMENTS"] > 4000).astype(int)
data.head()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE,HIGH_PAYMENT
0,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2.0,1000.0,201.802084,139.509787,0.000000,12,0
1,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0.0,7000.0,4103.032597,1072.340217,0.222222,12,1
2,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12.0,7500.0,622.066742,627.284787,0.000000,12,0
3,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1.0,7500.0,0.000000,312.343947,0.000000,12,0
4,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1.0,1200.0,678.334763,244.791237,0.000000,12,0


## Question 2 Supervised Learning with KNN and Ensemble Models Now that your dataset is cleaned and labeled, you will try to predict whether a customer will make a high payment using supervised learning.

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import VotingClassifier

2.1 Distance-Based Learning Normalize your features and explain why this step is important for KNN.



Train a K-Nearest Neighbors classifier to predict HIGH_PAYMENT.



Pick a reasonable k. (restrict to only 3-5 , you can try elbow method , it will not work well in this case), Justify your pick.



Evaluate your model using a confusion matrix and performance metrics: Accuracy, Precision, Recall, F1-score.



Try computing both Euclidean and Cosine distance manually for a small sample of data points.

In [12]:
# We are predicting HIGH_PAYMENTS
# It's reasonable to assume we do not have PAYMENTS to begin with
X = data.select_dtypes(include='number').drop(columns=["HIGH_PAYMENT", "PAYMENTS"])
y = data["HIGH_PAYMENT"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

y_pred = knn.predict(X_test_scaled)

In [14]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[1585   20]
 [ 103   82]]


In [15]:
print("Classification Report:")
print(classification_report(y_test, y_pred, digits=3))

Classification Report:
              precision    recall  f1-score   support

           0      0.939     0.988     0.963      1605
           1      0.804     0.443     0.571       185

    accuracy                          0.931      1790
   macro avg      0.871     0.715     0.767      1790
weighted avg      0.925     0.931     0.922      1790



In [16]:
report = classification_report(y_test, y_pred, output_dict=True)
accuracy = report['accuracy']
precision = report['weighted avg']['precision']
recall = report['weighted avg']['recall']
f1 = report['weighted avg']['f1-score']

print(f"Accuracy : {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1-score : {f1:.3f}")

Accuracy : 0.931
Precision: 0.925
Recall   : 0.931
F1-score : 0.922


## The accuracy is decent, but it alone doesn't tell the full story
## For the high payment case, the model only has 0.443 recall. The majority of high payers are misclassified as low
## Macro AVG for f-1 score is decent, but it's misleading as low payments dominate the dataset

In [20]:
for _ in range(3):
    idx1, idx2 = random.sample(range(X_train_scaled.shape[0]), 2)

    point1 = X_train_scaled[idx1]
    point2 = X_train_scaled[idx2]

    euclidean = np.linalg.norm(point1 - point2)
    cos_sim = cosine_similarity([point1], [point2])[0][0]
    cos_dist = 1 - cos_sim

    print(f"index {idx1} vs {idx2}")
    print(f"  Euclidean distance : {euclidean:.4f}")
    print(f"  Cosine similarity   : {cos_sim:.4f}")
    print(f"  Cosine distance     : {cos_dist:.4f}\n")

index 1926 vs 2525
  Euclidean distance : 5.9926
  Cosine similarity   : -0.3784
  Cosine distance     : 1.3784

index 3936 vs 747
  Euclidean distance : 5.0402
  Cosine similarity   : -0.2807
  Cosine distance     : 1.2807

index 63 vs 3866
  Euclidean distance : 2.4607
  Cosine similarity   : 0.6101
  Cosine distance     : 0.3899



2.2 Linear Discriminant Analysis (LDA)

Explain the concept of LDA and scatter matrices.



Apply LDA to reduce your data to justifyable dimensions.



Visualize the result and interpret the separation between classes.

In [79]:
# Binary Classification. There can only be one dimension
lda = LinearDiscriminantAnalysis(n_components = 1)
X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

In [80]:
lda_df = pd.DataFrame({
    "LDA1": X_test_lda[:, 0],
    "HIGH_PAYMENT": y_test.values
})

fig = px.histogram(
    lda_df,
    x="LDA1",
    color="HIGH_PAYMENT",
    nbins=30,
    barmode="overlay",
    histnorm="probability density",
    title="LDA Projection of Test Set",
    color_discrete_sequence=["#2ca02c", "#d62728"]
)
fig.update_layout(xaxis_title="LDA Component", yaxis_title="Density")
fig.show()

## If we were to separate at 0, then we have classified the majority of high_payment and low_payment to decent accuracy. There is still this sizeable overlap, for which a projection method will be unable to classify correctly. Tail separation is very good

2.3 Ensemble Learning Train two different classifiers:

Logistic Regression – adjust the regularization rates (try values like 0.01, 0.1, 1, 10).

SGDClassifier (Stochastic Gradient Descent Classifier) – experiment with different learning rates (eta0) and regularization penalties (l2, l1, etc.)

Suggested settings to try:

penalty = 'l2', eta0 = 0.01, 0.1, 1


Hint: If you are comfortable using third-party packages, you may optionally try XGBoostClassifier and explore learning_rate and reg_lambda.

Combine the two models using a simple ensemble method (e.g., majority voting).



In [93]:
C_values = [0.01, 0.1, 0.5, 1, 5, 10]
scores_LR = {}

for C in C_values:
    clf = LogisticRegression(C=C, random_state = 69)
    clf.fit(X_train_lda, y_train)
    y_pred = clf.predict(X_test_lda)
    score = f1_score(y_test, y_pred)
    scores_LR[C] = score

best_c = max(scores_LR, key=scores_LR.get)
print(best_c)

1


In [94]:
best_model = LogisticRegression(C = best_c, random_state = 69)
best_model.fit(X_train_lda, y_train)
y_pred = best_model.predict(X_test_lda)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[1581   24]
 [ 100   85]]


In [95]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))


Classification Report:
              precision    recall  f1-score   support

           0      0.941     0.985     0.962      1605
           1      0.780     0.459     0.578       185

    accuracy                          0.931      1790
   macro avg      0.860     0.722     0.770      1790
weighted avg      0.924     0.931     0.923      1790



In [96]:
report = classification_report(y_test, y_pred, output_dict=True)
accuracy = report['accuracy']
precision = report['weighted avg']['precision']
recall = report['weighted avg']['recall']
f1 = report['weighted avg']['f1-score']

print(f"Accuracy : {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1-score : {f1:.3f}")

Accuracy : 0.931
Precision: 0.924
Recall   : 0.931
F1-score : 0.923


## For high-payment class, the model is equally as bad as KNN. Only 46% of all high payers are identified.
## The overall performance is still decent, as the dataset is dominated by low payers
## Overall, Logistic Regression performs very close to KNN

In [97]:
penalties = ['l2']
etas = [0.01, 0.05, 0.1, 0.5, 1]
scores_SGD = {}

for penalty in penalties:
    for eta in etas:
        clf = SGDClassifier(loss='log_loss', penalty=penalty, eta0=eta, learning_rate='constant', max_iter=1000, random_state=69)
        clf.fit(X_train_lda, y_train)
        y_pred = clf.predict(X_test_lda)
        score = f1_score(y_test, y_pred)
        scores_SGD[(penalty, eta)] = score

best_params = max(scores_SGD, key=scores_SGD.get)
best_params[1]

0.1

In [98]:
best_penalty, best_eta = best_params

best_model = SGDClassifier(loss='log_loss', penalty=best_penalty, eta0=best_eta, learning_rate='constant', max_iter=1000, random_state=69)
best_model.fit(X_train_lda, y_train)
y_pred = best_model.predict(X_test_lda)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[1556   49]
 [  82  103]]


In [99]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))


Classification Report:
              precision    recall  f1-score   support

           0      0.950     0.969     0.960      1605
           1      0.678     0.557     0.611       185

    accuracy                          0.927      1790
   macro avg      0.814     0.763     0.785      1790
weighted avg      0.922     0.927     0.924      1790



In [100]:
report = classification_report(y_test, y_pred, output_dict=True)
accuracy = report['accuracy']
precision = report['weighted avg']['precision']
recall = report['weighted avg']['recall']
f1 = report['weighted avg']['f1-score']

print(f"Accuracy : {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1-score : {f1:.3f}")

Accuracy : 0.927
Precision: 0.922
Recall   : 0.927
F1-score : 0.924


## The classification within high payers is significantly more accurate than previous methods
## The same accuracy across the board. With better performance on high_payment = 1, I would say SGD is overall superior than the previous two

Evaluate the performance of each model and the ensemble using:

Accuracy, Precision, Recall, F1-score



In [101]:
log_reg = LogisticRegression(C=best_c, random_state=42)
sgd = SGDClassifier(loss='log_loss', penalty=best_penalty, eta0=best_eta, learning_rate='constant', max_iter=1000, random_state=69)

ensemble = VotingClassifier(
    estimators=[
        ('log_reg', log_reg),
        ('sgd', sgd)
    ],
    voting='soft'
)

In [102]:
ensemble.fit(X_train_lda, y_train)
y_pred = ensemble.predict(X_test_lda)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[1571   34]
 [  91   94]]


In [103]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))


Classification Report:
              precision    recall  f1-score   support

           0      0.945     0.979     0.962      1605
           1      0.734     0.508     0.601       185

    accuracy                          0.930      1790
   macro avg      0.840     0.743     0.781      1790
weighted avg      0.923     0.930     0.924      1790



In [104]:
report = classification_report(y_test, y_pred, output_dict=True)
accuracy = report['accuracy']
precision = report['weighted avg']['precision']
recall = report['weighted avg']['recall']
f1 = report['weighted avg']['f1-score']

print(f"Accuracy : {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"F1-score : {f1:.3f}")

Accuracy : 0.930
Precision: 0.923
Recall   : 0.930
F1-score : 0.924


## The result is unremarkable. Since the overall performance of all models are roughly the same, I want to focus on the high_payment = 1 case
## All the evaluative metrics of this simple ensemble method ly between Logistic and SGD. It's a balanced version between the two, but there is no clear advantage compared to either model individually

Briefly describe:

How changes in regularization and learning rate affected model performance.



In [105]:
print(scores_LR)
print(scores_SGD)

{0.01: 0.558303886925795, 0.1: 0.5733788395904437, 0.5: 0.5733788395904437, 1: 0.5782312925170068, 5: 0.5782312925170068, 10: 0.5782312925170068}
{('l2', 0.01): 0.581081081081081, ('l2', 0.05): 0.5911949685534591, ('l2', 0.1): 0.6112759643916914, ('l2', 0.5): 0.5733788395904437, ('l2', 1): 0.5342960288808665}


In [109]:
df_lr = pd.DataFrame({'C': list(scores_LR.keys()), 'F1-score': list(scores_LR.values())})

fig_lr = px.line(df_lr, x='C', y='F1-score', markers=True, title='Logistic Regression: F1-score vs C')
fig_lr.update_layout(xaxis_type='log', xaxis_title='C (Regularization Strength)', yaxis_title='F1-score')
fig_lr.show()

etas = [key[1] for key in scores_SGD.keys()]
f1_scores = list(scores_SGD.values())

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=etas,
    y=f1_scores,
    mode='lines+markers',
    name='SGDClassifier'
))

fig.update_layout(
    title='SGDClassifier: F1-score vs eta0',
    xaxis_title='eta0 (Learning Rate)',
    yaxis_title='F1-score',
    xaxis_type='log',
    template='plotly_white'
)

fig.show()

## For the regularization coefficient, it appears the model performance increases w.r.t. the coefficient, but does not increase further after lambda = 1
## For the learning rate, there is one parameter value that results in the best F-1 score amongst the values I tested

Which combination performed best, and why you think that happened.

In [110]:
logistic_Cs = [0.01, 0.1, 0.5, 1, 5, 10]
sgd_settings = [('l2', 0.01), ('l2', 0.05), ('l2', 0.1), ('l2', 0.5), ('l2', 1)]

ensemble_scores = {}
for C in logistic_Cs:
    for penalty, eta in sgd_settings:
        log_reg = LogisticRegression(C=C, random_state=42, max_iter=1000)
        sgd = SGDClassifier(loss='log_loss', penalty=penalty, eta0=eta,
                            learning_rate='constant', max_iter=1000, random_state=42)

        ensemble = VotingClassifier(
            estimators=[('log', log_reg), ('sgd', sgd)],
            voting='soft'
        )

        ensemble.fit(X_train_lda, y_train)
        y_pred = ensemble.predict(X_test_lda)
        score = f1_score(y_test, y_pred)

        ensemble_scores[(C, eta)] = score

best_params = max(ensemble_scores, key=ensemble_scores.get)
best_score = ensemble_scores[best_params]
print(best_params)

(0.01, 1)


## The grid search results in a different regularization coefficient
## A regularization of at least 0.01 result in the best performance, suggesting that the model benefits from limiting the coefficient values
## I am not sure why eta0 = 1 results in the best fit. Maybe it's simply the behavior of SGD, that both learning too fast or two slow is suboptimal for the model

## Question 3 Unsupervised Learning and Dimensionality Reduction Your manager wants you to group customers based on their behavior and understand patterns without relying on labels.

In [115]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

3.1 Principal Component Analysis (PCA) Normalize the data.

Apply PCA and report the explained variance of the first 3 components.



In [112]:
X = data.select_dtypes(include='number').drop(columns=['HIGH_PAYMENT', 'PAYMENTS'], errors='ignore')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

In [113]:
explained_var = pca.explained_variance_ratio_
print(f"Explained Variance of PC1: {explained_var[0]:.4f}")
print(f"Explained Variance of PC2: {explained_var[1]:.4f}")
print(f"Explained Variance of PC3: {explained_var[2]:.4f}")
print(f"Total (PC1–PC3): {explained_var[:3].sum():.4f}")

Explained Variance of PC1: 0.3003
Explained Variance of PC2: 0.2176
Explained Variance of PC3: 0.0891
Total (PC1–PC3): 0.6070


Visualize the data in the PCA space (3D plot).



In [114]:
pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
pca_df['HIGH_PAYMENT'] = data['HIGH_PAYMENT'].values

fig = px.scatter_3d(
    pca_df, x='PC1', y='PC2', z='PC3',
    color='HIGH_PAYMENT',
    title='3D PCA Projection',
    labels={'HIGH_PAYMENT': 'High Payment Class'}
)
fig.show()

Interpret whether the data is well-represented in the new space.

## There appears to be some useful trend observed in the separation, but the overlapping region is significant. I wouldn't consider a good transformer representation

3.2 K-Means Clustering Use the PCA-transformed data. Plot the Elbow Curve for k values from 2 to 10 using both random and kmeans++ initialization. find the optimal K.



In [116]:
inits = ['random', 'k-means++']
ks = range(2, 11)
inertia_results = {init: [] for init in inits}

for init in inits:
    for k in ks:
        kmeans = KMeans(n_clusters=k, init=init, n_init=10, random_state=42)
        kmeans.fit(X_pca)
        inertia_results[init].append(kmeans.inertia_)

In [117]:
fig = go.Figure()
for init in inits:
    fig.add_trace(go.Scatter(x=list(ks), y=inertia_results[init],
                             mode='lines+markers',
                             name=f'Init: {init}'))

fig.update_layout(title='Elbow Curve for K-Means (PCA Space)',
                  xaxis_title='Number of Clusters (k)',
                  yaxis_title='Inertia (Within-Cluster Sum of Squares)',
                  template='plotly_white')
fig.show()

## The elbow occurrs at k = 4, after which the within cluster inertia reduction slows down
## There's absolutely no difference between random and k-means++

Fit the final model and visualize the clusters in 3D.



In [119]:
kmeans_final = KMeans(n_clusters=4, init='k-means++', n_init=10, random_state=42)
labels_pca = kmeans_final.fit_predict(X_pca)

pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2', 'PC3'])
pca_df['Cluster'] = labels_pca

fig = px.scatter_3d(
    pca_df, x='PC1', y='PC2', z='PC3',
    color=pca_df['Cluster'].astype(str),
    title='K-Means Clustering (k = 4) in PCA-Reduced Space',
    labels={'Cluster': 'Cluster ID'}
)
fig.show()

Repeat clustering on the original (non-PCA) data. Choose 5 features and plot their distributions cluster by cluster.



In [120]:
selected_features = ['PURCHASES', 'BALANCE', 'CREDIT_LIMIT', 'PRC_FULL_PAYMENT', 'PURCHASES_FREQUENCY']

X_selected = data[selected_features]
X_scaled = StandardScaler().fit_transform(X_selected)

kmeans_raw = KMeans(n_clusters=4, init='k-means++', n_init=10, random_state=42)
clusters_raw = kmeans_raw.fit_predict(X_scaled)

data_clustered = data.copy()
data_clustered['Cluster'] = clusters_raw

for feature in selected_features:
    fig = px.box(data_clustered, x='Cluster', y=feature, color='Cluster',
                 title=f'Distribution of {feature} by Cluster (Original Features)')
    fig.show()

Compare clustering results before and after PCA. Which works better and why?

In [121]:
kmeans_pca = KMeans(n_clusters=4, init='k-means++', n_init=10, random_state=42)
pca_cluster_labels = kmeans_pca.fit_predict(X_pca)

data_pca_clustered = data.copy()
data_pca_clustered['PCA_Cluster'] = pca_cluster_labels

selected_features = ['PURCHASES', 'BALANCE', 'CREDIT_LIMIT', 'PRC_FULL_PAYMENT', 'PURCHASES_FREQUENCY']

for feature in selected_features:
    fig = px.box(data_pca_clustered, x='PCA_Cluster', y=feature, color='PCA_Cluster',
                 title=f'Distribution of {feature} by PCA Cluster')
    fig.show()

## For PURCHASES and PURCHASES_FREQUENCY, the feature separation is clearer after PCA
## For BALANCE and PRC_FULL_PAYMENT, the feature separation is worse with PCA
## Overall, there is not a clear winner between K_means before or after PCA